In [1]:
import syft as sy


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node = sy.orchestra.launch(name="test-jobs", dev_mode=True, reset=True, create_producer=True, queue_port=5556, n_consumers=3)
client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Logged into <test-jobs: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [3]:
x = sy.ActionObject.from_obj([1,2,3])
x_ptr = x.send(client)

In [4]:
@sy.syft_function()
def process_batch(batch):
    # takes 30 hours normally
    print(f"starting batch {batch}")
    from time import sleep
    import random
    sleep(random.random()*5)
    print("done")
    return batch + 1

SyftSuccess: Syft function 'process_batch' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [5]:
client.code.submit(process_batch)

SyftSuccess: User Code Submitted

In [6]:
@sy.syft_function()
def aggregate_job(job_results):
    results = [job.resolve.get() for job in job_results]
    print([(res, type(res))for res in results])
    return sum(results)

SyftSuccess: Syft function 'aggregate_job' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [7]:
client.code.submit(aggregate_job)

SyftSuccess: User Code Submitted

In [8]:
@sy.syft_function_single_use(x=x_ptr)
def process_all(domain, x):
    jobs = []
    print("Launching jobs")
    for elem in x:
        # We inject a domain object in the scope
        batch_job = domain.launch_job(process_batch, batch=elem)
        jobs.append(batch_job)
    print("starting aggregation")
    print("Done")
    return domain.launch_job(aggregate_job, job_results=jobs)

SyftSuccess: Syft function 'process_all' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [9]:
client.code.request_code_execution(process_all)
client.requests[-1].approve()

Request approved for domain test-jobs


SyftSuccess: Request 25199588a43047549635f7fbe20ac3df changes applied

In [10]:
from time import sleep
job = client.code.process_all(x=x_ptr, blocking=False)
# sleep(10)

In [12]:
job.subjobs[2].result.get()

Kwargs: {'x': Pointer:
[1, 2, 3], 'uid': <UID: fe0529b5e3264a769037ce7390e717fa>, 'communication_protocol': 'dev'}
Produder args: True
Produder kwargs: True


[<QueueItem: 89430ba9dfa447b78176f1190c410f0c>: Status.CREATED]


IndexError: list index out of range

13/11/23 11:17:04 FUNCTION LOG (28b4d5c6cd0f4dfaa4b3bb1df212eb96): Launching jobs
Kwargs: {'batch': <UID: 9e7fd97ff9b54b8aa089101536f6d82c>}
Produder args: True
Produder kwargs: True
Kwargs: {'batch': <UID: ffc1c09accaf4f8f85e1a668af2a08b8>}
Produder args: True
Produder kwargs: True


[<QueueItem: 89430ba9dfa447b78176f1190c410f0c>: Status.PROCESSING, <QueueItem: 21faa050231740f799cf7c7305408cb2>: Status.CREATED, <QueueItem: 409308f2c83647119f64b5181da53886>: Status.CREATED]
[<QueueItem: 89430ba9dfa447b78176f1190c410f0c>: Status.PROCESSING, <QueueItem: 21faa050231740f799cf7c7305408cb2>: Status.PROCESSING, <QueueItem: 409308f2c83647119f64b5181da53886>: Status.CREATED]


...

13/11/23 11:17:08 FUNCTION LOG (02e68e33a3ed404e99b61c35c8e4606a): done
Kwargs: {'job_results': <UID: 6e1a3916ae2b41159db51ac3d9220be8>}
JOBS: [SyftNotReady: 94188b058ab743f2beea9d1e8ae5513e not ready yet., Pointer:
None, Pointer:
None]
Produder args: True
Produder kwargs: False


[<QueueItem: 89430ba9dfa447b78176f1190c410f0c>: Status.COMPLETED, <QueueItem: 21faa050231740f799cf7c7305408cb2>: Status.PROCESSING, <QueueItem: 409308f2c83647119f64b5181da53886>: Status.COMPLETED, <QueueItem: 0165118aaee94a58827fb91e51434201>: Status.COMPLETED, <QueueItem: 9b43523bd3fc44b2a4bdd5003b22d077>: Status.CREATED]


13/11/23 11:17:09 FUNCTION LOG (94188b058ab743f2beea9d1e8ae5513e): done
Kwargs: {'job_results': <UID: 6e1a3916ae2b41159db51ac3d9220be8>}
JOBS: [Pointer:
None, Pointer:
None, Pointer:
None]
Produder args: True
Produder kwargs: True


[<QueueItem: 89430ba9dfa447b78176f1190c410f0c>: Status.COMPLETED, <QueueItem: 21faa050231740f799cf7c7305408cb2>: Status.COMPLETED, <QueueItem: 409308f2c83647119f64b5181da53886>: Status.COMPLETED, <QueueItem: 0165118aaee94a58827fb91e51434201>: Status.COMPLETED, <QueueItem: 9b43523bd3fc44b2a4bdd5003b22d077>: Status.CREATED]


13/11/23 11:17:10 FUNCTION LOG (c4ca3acad9764303a6c9d5eb8d7fb365): [(2, <class 'int'>), (3, <class 'int'>), (4, <class 'int'>)]


In [33]:
result = job.result.get()

In [36]:
result.fetch()

In [40]:
result.result.get()

9